In [1]:
import pygsheets
from tqdm import tqdm

In [2]:
client = pygsheets.authorize()

### Indonesian Data

In [26]:
id_to_key_ref_ind = {
    'W1': ''
}
id_to_key_pred_ind = {
}

In [27]:
# Open worksheet
sh_ref = client.open_by_key(id_to_key_ref_ind['W1'])
wks_ref = sh_ref[1]
sh_pred = client.open_by_key(id_to_key_pred_ind['W6'])
wks_pred = sh_pred[0]

# Get answers
ref_ans = wks_ref.get_col(6)[1:]
pred_ans = wks_pred.get_col(6)[1:]

# Check answer consistency
is_consistent = []
const_temp = []
rows_idx = []
idx = 1
for pred, ref in tqdm(zip(pred_ans, ref_ans)):
    idx += 1
    consistency = 'TRUE' if pred == ref else 'FALSE'
    const_temp.append(consistency)

    if len(const_temp) == 5:
        if 'FALSE' in const_temp:
            const_temp[0] = 'FALSE'
        for const in const_temp:
            is_consistent.append(const)
        const_temp = []
        rows_idx.append(('H' + str(idx-4), 'H' + str(idx)))

# Add new colomn & update its values
wks_pred.insert_cols(6, 2)
wks_pred.adjust_column_width(7, 8, pixel_size=100)
wks_pred.set_data_validation('G2', 'G1251', condition_type='BOOLEAN')
wks_pred.set_data_validation('H2', 'H1251')
wks_pred.update_col(7, ['Gold Answer'] + ref_ans)
wks_pred.update_col(8, ['Is Consistent'] + is_consistent)

1250it [00:00, 309698.15it/s]


In [24]:
for idx_start, idx_end in tqdm(rows_idx[93:]):
    rng = wks_pred.get_values(idx_start, idx_end, returnas='range')
    rng.merge_cells()

  0%|          | 0/157 [00:00<?, ?it/s]

100%|██████████| 157/157 [04:02<00:00,  1.55s/it]


In [25]:
wks_pred.add_conditional_formatting(start='H2', 
                                     end='H1251', 
                                     condition_type='TEXT_CONTAINS', 
                                     format={'backgroundColor': {'red': 0.957, 'green': 0.8, 'blue': 0.8, 'alpha': 1}}, 
                                     condition_values=['FALSE'])

### Sundanese Data

In [62]:
id_to_key_ref_sun = {
    'W1': '',
    'W2': ''
}
id_to_key_pred_sun = {
    'W1': '',
    'W2': '',
    'W3': '',
    'W4': '',
    'W5': '',
    'W6': '',
}

In [63]:
# Open worksheet
sh_ref = client.open_by_key(id_to_key_ref_sun['W1'])
wks_ref = sh_ref[1]
sh_pred = client.open_by_key(id_to_key_pred_sun['W6'])
wks_pred = sh_pred[0]

# Get answers
ref_ans = wks_ref.get_col(7)[1:]
pred_ans = wks_pred.get_col(7)[1:]

# Check answer consistency
is_consistent = []
const_temp = []
rows_idx = []
idx = 1
for pred, ref in tqdm(zip(pred_ans, ref_ans)):
    idx += 1
    consistency = 'TRUE' if pred == ref else 'FALSE'
    const_temp.append(consistency)

    if len(const_temp) == 5:
        if 'FALSE' in const_temp:
            const_temp[0] = 'FALSE'
        for const in const_temp:
            is_consistent.append(const)
        const_temp = []
        rows_idx.append(('I' + str(idx-4), 'I' + str(idx)))

# Add new colomn & update its values
wks_pred.insert_cols(7, 2)
wks_pred.adjust_column_width(8, 9, pixel_size=100)
wks_pred.set_data_validation('H2', 'H1251', condition_type='BOOLEAN')
wks_pred.set_data_validation('I2', 'I1251')
wks_pred.update_col(8, ['Gold Answer'] + ref_ans)
wks_pred.update_col(9, ['Is Consistent'] + is_consistent)

1250it [00:00, 305867.80it/s]


In [64]:
for idx_start, idx_end in tqdm(rows_idx):
    rng = wks_pred.get_values(idx_start, idx_end, returnas='range')
    rng.merge_cells()

100%|██████████| 250/250 [04:11<00:00,  1.01s/it]


In [65]:
wks_pred.add_conditional_formatting(start='I2', 
                                     end='I1251', 
                                     condition_type='TEXT_CONTAINS', 
                                     format={'backgroundColor': {'red': 0.957, 'green': 0.8, 'blue': 0.8, 'alpha': 1}}, 
                                     condition_values=['FALSE'])